# Introduction to Agents in Databricks

Let's create an agent, __RefundAgent__ that identifies orders that are eligible for a refund based on the time it took to deliver the order.

To do this we'll need to equip our agent with two tools:

  1. __Order Details tool__: This tool will take an `order_id` as an input and return all the events associated with this order
  2. __Location Timing tool__: This tool will take a `location` and retrieve the 50/75/99 percentile delivery times for that specific location

We'll attach these tools to a tool capable LLM like `llama-3-3` in the Playground and test it out directly in the notebook.

## Project Overview

We will work through the following steps:   
1. Initialize the catalog and volume and set up the data
2. Define tools for the agent to use
3. Use the playground to call the tools and test the agent
4. Export the agent from the playground to a notebook

This repo includes a zipped data file that contains two weeks of sample delivery data. We will start by unzipping the file and loading the data into a Delta table.

### Setup

In [0]:
from utils.utils import (
    setup_catalog_and_volume,
    copy_raw_data_to_volume,
    initialize_events_table,
    drop_gk_demo_catalog,
    initialize_order_delivery_times_view,
)

# Drop existing catalog/volume/table if you need to start fresh
# drop_gk_demo_catalog(spark)

## 1. Setup the catalog and volume
setup_catalog_and_volume(spark)

## 2. Copy the raw data to the volume
copy_raw_data_to_volume()

## 3. Initialize the events table and order timings view
initialize_events_table(spark)
initialize_order_delivery_times_view(spark)

## Register Tools

We will create two [Unity Catalog functions](https://docs.databricks.com/aws/en/generative-ai/agent-framework/create-custom-tool) that will be used as tools for the agent.

### Order Details Tool

In [0]:
%sql
CREATE OR REPLACE FUNCTION gk_demo.default.get_order_details(oid STRING)
RETURNS TABLE (
  body STRING COMMENT 'Body of the event',
  event_type STRING COMMENT 'The type of event',
  order_id STRING COMMENT 'The order id',
  ts STRING COMMENT 'The timestamp of the event',
  location STRING COMMENT 'the location of the order'
)
COMMENT 'Returns all events associated with the order id (oid)'
RETURN
  SELECT body, event_type, order_id, ts, location
  FROM gk_demo.default.all_events ae
  WHERE ae.order_id = oid;

### Order Timings Tool

This tool will return the 50/75/99th percentile of total delivery times for a given location.

In [0]:
%sql
CREATE OR REPLACE FUNCTION gk_demo.default.get_location_timings(loc STRING COMMENT 'Location name as a string')
RETURNS TABLE (
  location STRING COMMENT 'Location of the order source',
  P50 FLOAT COMMENT '50th percentile',
  P75 FLOAT COMMENT '75th percentile',
  P99 FLOAT COMMENT '99th percentile'
)
COMMENT 'Returns the 50/75/99th percentile of total delivery times for locations'
RETURN
  SELECT location, P50, P75, P99
  FROM gk_demo.default.order_delivery_times_per_location_view AS odlt
  WHERE odlt.location = loc;

## Try out the Agent in the Playground

1. Copy this system prompt
    ```
    You are RefundGPT, a CX agent responsible for refunding food delivery orders.

    You can call tools to gather the information you need. Start with an `order_id`.

    Instructions:
    1. Call `order_details(order_id)` first to get event history.
    2. Figure out the delivery duration by comparing timestamps in the events.
    3. Extract the location (either directly or from the first event's body).
    4. Call `get_location_timings(location)` to get the P50/P75/P99 values.
    5. Compare actual delivery time to those percentiles to decide on a fair refund.

    Output a single-line JSON with these fields:
    - `refund_usd` (float),
    - `refund_class` ("none" | "partial" | "full"),
    - `reason` (short human explanation)

    You must return only the JSON. No extra text or markdown.
    ```
2. Open Playground from the left sidebar
3. Paste the system prompt and select an LLM with tool calling capabilities (`Meta Llama 3.3 70B Instruct`)

<img src="./images/agents/playground_add_tools.png" width="75%"/>

4. Try an the following order_id: `04e9a339e7fb4435b5a084a60edd927f` as an input to see the agent in action. Below we see the agent called both tools and returned a JSON object with a decision about whether the timing data of this order makes this order eligible for a refund.

<img src="./images/agents/run_agent_in_notebook.png" width="75%">

5. We can export our prototyped agent using __Create Agent Notebook__.

<img src="./images/agents/create_agent_notebook.png" width="75%">

6. Follow the remaining steps in the generated notebook on defining agents in code, managing experiments and offline evals, deploying, and online monitoring.